In [1]:
#Importing of relavent functions, pre-trained models, and libraries
import string
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

def l1_distance(vectors):
    x, y = vectors
    return tf.reduce_sum(tf.abs(x - y), axis=1, keepdims=True)

import gensim.downloader
import numpy as np
from gensim.models import Word2Vec
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.losses import cosine_similarity
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Lambda, Dense
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Embedder = gensim.downloader.load('word2vec-google-news-300')

In [2]:
#Reading in the training and test sets
Master = pd.read_csv("QQP/train.tsv",sep="\t")
Train,Dev = train_test_split(Master,test_size=0.1,random_state=42)
Test = pd.read_csv("QQP/dev.tsv",sep="\t")

In [3]:
#Pre-processing of training set

Train = Train.dropna()
Train = Train[~Train.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Train["question1"])):
    Train["question1"].iloc[i] = remove_punctuation(Train["question1"].iloc[i].lower())
    Train["question2"].iloc[i] = remove_punctuation(Train["question2"].iloc[i].lower())

Train = Train.dropna()
Train = Train[~Train.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

Q1 = Train["question1"].copy()
Q2 = Train["question2"].copy()

#Tokenization of training set
Q1_Tokens = Q1.copy()
Q2_Tokens = Q2.copy()
for i in range(len(Q1_Tokens)):
    Q1_Tokens.iloc[i] = Q1_Tokens.iloc[i].split()
    Q2_Tokens.iloc[i] = Q2_Tokens.iloc[i].split()

#Embedding of training set
Q1_Embedded = Q1_Tokens.copy()
Q2_Embedded = Q2_Tokens.copy()
for i in range(len(Q1_Embedded)):
  Sum_Q1 = np.zeros(300)
  Counter_Q1 = 0
  for word in Q1_Embedded.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + Embedder[word]
      Counter_Q1 = Counter_Q1 + 1
  Q1_Embedded.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  Counter_Q2 = 0
  for word in Q2_Embedded.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + Embedder[word]
      Counter_Q2 = Counter_Q2 + 1
  Q2_Embedded.iloc[i] = Sum_Q2


#Formatting inputs to match model's input format
Q1_Inputs = []
for i in range(len(Q1_Embedded)):
    #print(i)
    Q1_Inputs.append(Q1_Embedded.iloc[i].tolist())
Q1_Inputs = np.array(Q1_Inputs)

Q2_Inputs = []
for i in range(len(Q2_Embedded)):
    Q2_Inputs.append(Q2_Embedded.iloc[i].tolist())
Q2_Inputs = np.array(Q2_Inputs)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [4]:
#Pre-Processing of validation set
Dev = Dev.dropna()
Dev = Dev[~Dev.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Dev["question1"])):
    Dev["question1"].iloc[i] = remove_punctuation(Dev["question1"].iloc[i].lower())
    Dev["question2"].iloc[i] = remove_punctuation(Dev["question2"].iloc[i].lower())

Dev = Dev.dropna()
Dev = Dev[~Dev.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

Q1_Dev = Dev["question1"].copy()
Q2_Dev = Dev["question2"].copy()

#Tokenization of validation set
Q1_Dev_Tokens = Q1_Dev.copy()
Q2_Dev_Tokens = Q2_Dev.copy()
for i in range(len(Q1_Dev_Tokens)):
    Q1_Dev_Tokens.iloc[i] = Q1_Dev_Tokens.iloc[i].split()
    Q2_Dev_Tokens.iloc[i] = Q2_Dev_Tokens.iloc[i].split()

#Embedding of validation set
from gensim.models import Word2Vec
Q1_Embedded_Dev = Q1_Dev_Tokens.copy()
Q2_Embedded_Dev = Q2_Dev_Tokens.copy()

for i in range(len(Q1_Embedded_Dev)):
  Sum_Q1 = np.zeros(300)
  for word in Q1_Embedded_Dev.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + (Embedder[word])
  Q1_Embedded_Dev.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  for word in Q2_Embedded_Dev.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + (Embedder[word])
  Q2_Embedded_Dev.iloc[i] = Sum_Q2

#Formating embedding to match model's input format

Q1_Inputs_Dev = []
for i in range(len(Q1_Embedded_Dev)):
    Q1_Inputs_Dev.append(Q1_Embedded_Dev.iloc[i].tolist())
Q1_Inputs_Dev = np.array(Q1_Inputs_Dev)

Q2_Inputs_Dev = []
for i in range(len(Q2_Embedded_Dev)):
    Q2_Inputs_Dev.append(Q2_Embedded_Dev.iloc[i].tolist())
Q2_Inputs_Dev = np.array(Q2_Inputs_Dev)

In [5]:
#Pre-processing the test set
Test = Test.dropna()
Test = Test[~Test.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

for i in range(len(Test["question1"])):
    Test["question1"].iloc[i] = remove_punctuation(Test["question1"].iloc[i].lower())
    Test["question2"].iloc[i] = remove_punctuation(Test["question2"].iloc[i].lower())

Test = Test.dropna()
Test = Test[~Test.apply(lambda row: row.str.strip().str.len().eq(0).any(), axis=1)]

#Tokenization of the test set
Q1_Test = Test["question1"].copy()
Q2_Test = Test["question2"].copy()

Q1_Test_Tokens = Q1_Test.copy()
Q2_Test_Tokens = Q2_Test.copy()
for i in range(len(Q1_Test_Tokens)):
    Q1_Test_Tokens.iloc[i] = Q1_Test_Tokens.iloc[i].split()
    Q2_Test_Tokens.iloc[i] = Q2_Test_Tokens.iloc[i].split()

#Creating embeddings for the test set
from gensim.models import Word2Vec
Q1_Embedded_Test = Q1_Test_Tokens.copy()
Q2_Embedded_Test = Q2_Test_Tokens.copy()

for i in range(len(Q1_Embedded_Test)):
  Sum_Q1 = np.zeros(300)
  for word in Q1_Embedded_Test.iloc[i]:
    if word in Embedder:
      Sum_Q1 = Sum_Q1 + (Embedder[word])
  Q1_Embedded_Test.iloc[i] = Sum_Q1 
  Sum_Q2 = np.zeros(300)
  for word in Q2_Embedded_Test.iloc[i]:
    if word in Embedder:
      Sum_Q2 = Sum_Q2 + (Embedder[word])
  Q2_Embedded_Test.iloc[i] = Sum_Q2 

#Formatting the inputs to match the inputs of the model
Q1_Inputs_Test = []
for i in range(len(Q1_Embedded_Test)):
    Q1_Inputs_Test.append(Q1_Embedded_Test.iloc[i].tolist())
Q1_Inputs_Test = np.array(Q1_Inputs_Test)

Q2_Inputs_Test = []
for i in range(len(Q2_Embedded_Test)):
    Q2_Inputs_Test.append(Q2_Embedded_Test.iloc[i].tolist())
Q2_Inputs_Test = np.array(Q2_Inputs_Test)

In [6]:
#Clear up ram as we don't need the embedder anymore
Embedder = 0

In [50]:
#Initialization of the Siamese network with twin multi-layer perceptron models
embedding_dim = 300

shared_network = keras.Sequential([
    layers.Dense(300, activation='relu', input_shape=(embedding_dim,)),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
    layers.Dense(512, activation='relu'),
])

early_stopping = EarlyStopping(
    monitor='val_accuracy', 
    patience=5,           
    restore_best_weights=True  
)

left_input = layers.Input(shape=(embedding_dim,))
right_input = layers.Input(shape=(embedding_dim,))

encoded_left = shared_network(left_input)
encoded_right = shared_network(right_input)

distance = Lambda(l1_distance)([encoded_left, encoded_right])
similarity_prediction = Dense(1, activation='sigmoid')(distance)

siamese_model = keras.Model(inputs=[left_input, right_input], outputs=similarity_prediction)

custom_learning_rate = 0.00007
custom_optimizer = Adam(learning_rate=custom_learning_rate)

siamese_model.compile(loss='binary_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])

In [51]:
#Training of the model

siamese_model.fit(
    [Q1_Inputs, Q2_Inputs],  # Your question embeddings
    np.array(Train["is_duplicate"]),  # Similarity labels (0 for dissimilar, 1 for similar)
    batch_size=64,
    epochs=25,
    validation_data=([Q1_Inputs_Dev, Q2_Inputs_Dev], np.array(Dev["is_duplicate"]))
    ,callbacks=[early_stopping]
)

Epoch 1/25
5117/5117 [==============================] - 28s 5ms/step - loss: 0.5810 - accuracy: 0.6721 - val_loss: 0.5239 - val_accuracy: 0.7182
Epoch 2/25
5117/5117 [==============================] - 29s 6ms/step - loss: 0.5082 - accuracy: 0.7388 - val_loss: 0.4923 - val_accuracy: 0.7562
Epoch 3/25
5117/5117 [==============================] - 29s 6ms/step - loss: 0.4731 - accuracy: 0.7670 - val_loss: 0.4729 - val_accuracy: 0.7750
Epoch 4/25
5117/5117 [==============================] - 29s 6ms/step - loss: 0.4424 - accuracy: 0.7885 - val_loss: 0.4619 - val_accuracy: 0.7809
Epoch 5/25
5117/5117 [==============================] - 28s 6ms/step - loss: 0.4148 - accuracy: 0.8069 - val_loss: 0.4452 - val_accuracy: 0.7912
Epoch 6/25
5117/5117 [==============================] - 29s 6ms/step - loss: 0.3882 - accuracy: 0.8226 - val_loss: 0.4372 - val_accuracy: 0.7976
Epoch 7/25
5117/5117 [==============================] - 29s 6ms/step - loss: 0.3632 - accuracy: 0.8381 - val_loss: 0.4365 - val_ac

In [52]:
#Predicting the training set
train_predictions = siamese_model.predict([Q1_Inputs, Q2_Inputs])
threshold = 0.5
train_binary_predictions = (train_predictions > threshold).astype(int)
training_accuracy = accuracy_score(Train["is_duplicate"].tolist(), train_binary_predictions)
print(f"Training Accuracy: {training_accuracy * 100:.2f}%")


10233/10233 [==============================] - 13s 1ms/step
Training Accuracy: 95.73%


In [53]:
#Predicting the validation set
Dev_predictions = siamese_model.predict([Q1_Inputs_Dev, Q2_Inputs_Dev])
threshold = 0.5
Dev_binary_predictions = (Dev_predictions > threshold).astype(int)
Dev_accuracy = accuracy_score(Dev["is_duplicate"].tolist(), Dev_binary_predictions)
print(f"Validation Accuracy: {Dev_accuracy * 100:.2f}%")

1137/1137 [==============================] - 1s 1ms/step
Validation Accuracy: 82.11%


In [54]:
#Predicting the test set
test_predictions = siamese_model.predict([Q1_Inputs_Test, Q2_Inputs_Test])
threshold = 0.5
test_binary_predictions = (test_predictions > threshold).astype(int)
test_accuracy = accuracy_score(Test["is_duplicate"].tolist(), test_binary_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


1264/1264 [==============================] - 2s 1ms/step
Test Accuracy: 81.73%


In [63]:
#Exploratory Data Analysis on training points that we failed to predict correctly

Train_Incorrect = []

for i in range(len(Train["is_duplicate"])):
    if Train["is_duplicate"].iloc[i] != train_binary_predictions[i]:
        Train_Incorrect.append(i)

Train_Incorrect = Train.iloc[Train_Incorrect,]

print(Train_Incorrect)

            id    qid1    qid2  \
244375  330930  457741  373850   
294890   24257   45345   45346   
22273   299055  421641  421642   
21037   173131  188364  267158   
13988    25920   48290   48291   
...        ...     ...     ...   
250396  304563  346385  427836   
82798   337217   23547  119932   
11534    37539   68294   68295   
341097   70696  121784  121785   
194027   47789   23696   85293   

                                                question1  \
244375  when did the columbia shuttle crew knew someth...   
294890  how do we differentiate fact from opinion on q...   
22273   what is the difference between a high function...   
21037      why do people who cant afford kids make babies   
13988   what hotel in ranipuram hillstation would be s...   
...                                                   ...   
250396       which computer language should i learn first   
82798   extraterrestrial life what is the most undenia...   
11534   what is the meaning of orthogonali

In [64]:
#Exploratory Data Analysis on validation points that we failed to predict correctly

Dev_Incorrect = []

for i in range(len(Dev["is_duplicate"])):
    if Dev["is_duplicate"].iloc[i] != Dev_binary_predictions[i]:
        Dev_Incorrect.append(i)

Dev_Incorrect = Dev.iloc[Dev_Incorrect,]

print(Dev_Incorrect)

            id    qid1    qid2  \
118406  184181   56834  281459   
300683  352765  481723   13812   
315399  173113  267132  267133   
235047  362101  276829  218965   
194785  268756  386370  243725   
...        ...     ...     ...   
76807   152811  123714  240002   
307126   98049  163001  163002   
338520  262289  378591  378592   
111220   67965   63559   36011   
176748  146762  231764  231765   

                                                question1  \
118406                examples of sole proprietorship the   
300683                do kashmiris wants to join pakistan   
315399  what do you see as the next big thing in robotics   
235047                      whats your best birthday gift   
194785  how is vinyl compared to cd in terms of sound ...   
...                                                   ...   
76807                 what is the reason for sleep apenea   
307126  what should someone know before moving to beng...   
338520                       how do you im

In [65]:
#Exploratory Data Analysis on test points that we failed to predict correctly

Test_Incorrect = []

for i in range(len(Test["is_duplicate"])):
    if Test["is_duplicate"].iloc[i] != test_binary_predictions[i]:
        Test_Incorrect.append(i)

Test_Incorrect = Test.iloc[Test_Incorrect,]

print(Test_Incorrect)

           id    qid1    qid2  \
2      172974  266948  175089   
3       15329   29298   29299   
6      144933   10927  229257   
8      382732  443182  514615   
19     308638  432498  432499   
...       ...     ...     ...   
40406  255959  370959  370960   
40419  214998  320750  320751   
40420  144862  229155  229156   
40423  146181  230973  230974   
40426  258093  373551  373552   

                                               question1  \
2           is there a reason why we should travel alone   
3      why are people so obsessed with having a girlf...   
6      what does a good answer on quora look like wha...   
8                  why is my life getting so complicated   
19     is it a bad time to buy a condo or a house in ...   
...                                                  ...   
40406  what type of spices go great with mashed potatoes   
40419        how can i really slow down my aging process   
40420                       do women cheat more than men   
404